<a href="https://colab.research.google.com/github/Zafar-Lab/scDREAMER/blob/main/2109_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" # difference between gpu '0' and '1'

"""
# for reproducibility with server...

os.environ["PYTHONHASHSEED"] = "0"
os.environ["TF_CUDNN_DETERMINISTIC"] = "1" # setting "true"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
"""

import numpy as np
import tensorflow as tf2
import random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import scipy.io
from sklearn.decomposition import PCA
import pdb

import pandas as pd
import scanpy as sc


import scipy.sparse
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from scipy import stats 
from scipy import * 
import datetime 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

os.getpid()


2023-09-25 15:18:03.543816: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 15:18:04.202863: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
non-resource variables are not supported in the long term


2529111

In [2]:

np.random.seed(0)
tf.set_random_seed(0)
random.seed(0)
tf2.random.set_seed(0)
tf2.keras.utils.set_random_seed(0)


In [3]:
import warnings
warnings.filterwarnings('ignore')
"""
!pip install -U scipy==1.5.0
!pip install scanpy==1.8.2
!pip install -U scikit-learn
"""

'\n!pip install -U scipy==1.5.0\n!pip install scanpy==1.8.2\n!pip install -U scikit-learn\n'

Building model

In [4]:

name = "Immune_Human"
name = "Pancreas"
name = "Human_Retina"
name = "Lung"
name = "Healthy_Heart"

name = "Human_Mouse"
name = "Lung"


In [5]:


data_path = {
"Immune_Human" : "/home/ajita/Documents/data_integration/Immune/Immune_ALL_human.h5ad",
"Pancreas" : "/home/ajita/Documents/data_integration/Pancreas/Pancreas.h5ad",
"Human_Mouse" : "/home/ajita/Documents/data_integration/hum_mou/hcl_mca_merged.h5ad",
"Human_Retina" : "/home/ajita/Documents/data_integration/Human_Retina/Human_Retina.h5ad",
"Lung" : "/home/ajita/Documents/data_integration/Lung/Lung_atlas_public.h5ad",
"Healthy_Heart" : "/home/ajita/Documents/data_integration/Healthy_Heart/Healthy_human_heart_adata.h5ad" 
}


batch_key_dict = {'Immune_Human' : 'batch',
                 'Lung' : 'batch',
                 'Pancreas' : 'tech',
                 'Human_Mouse' : 'batch',
                 'Human_Retina': "Batch",
                 "Healthy_Heart" : "sampleID"
                 
                }
cell_type_key_dict = {'Immune_Human' : 'final_annotation',
                 'Lung' : 'cell_type',  
                 'Pancreas' : 'celltype', 
                 'Human_Mouse' : "celltype", 
                 "Human_Retina":"Subcluster",
                "Healthy_Heart" : "celltype"
                    }    

shuffle_type_dict = {'Immune_Human' : 1, 'Lung' : 2, 'Pancreas' : 1, 'Human_Mouse' : 1, 
                 "Human_Retina":2,"Healthy_Heart" : 1
                    }
learning_rate = {
                 'Immune_Human' : {"lr_ae" : 0.0002, "lr_dis": 0.0007, "lr_bc": 0.0007},
                 'Lung' : {"lr_ae" : 0.0002 , "lr_dis": 0.0007, "lr_bc": 0.0007},
                 'Pancreas' : {"lr_ae" : 0.0002, "lr_dis": 0.0007, "lr_bc": 0.0007},
                 'Human_Retina': {"lr_ae" : 0.0002, "lr_dis": 0.0007, "lr_bc": 0.0007},
                 'Human_Mouse' : {"lr_ae" : 0.0001, "lr_dis": 0.00001, "lr_bc": 0.0007},
                 "Healthy_Heart" : {"lr_ae" : 0.0001, "lr_dis": 0.00001, "lr_bc": 0.0007}
}


# big datasets: 20 dimensions
latent_dim = {
                'Immune_Human' : 10,
                 'Lung' : 10,
                 'Pancreas' : 10,
                 'Human_Mouse' : 20,
                 'Human_Retina': 10,
                 "Healthy_Heart" : 20
    
            }
BS = {
    'Immune_Human' : 128,
     'Lung' : 128,
     'Pancreas' : 128,
     'Human_Mouse' : 128,
     'Human_Retina': 128,
     "Healthy_Heart" : 128
}

epoch_dict = {
    'Immune_Human' : 150,
     'Lung' : 250,
     'Pancreas' : 300,
     'Human_Mouse' : 200,
     'Human_Retina': 300,
     "Healthy_Heart" : 300
}

"""
Below variable takes care of sparse matrix
If 1: pd.DataFrame.sparse.from_spmatrix(Ann.X) is used
If 0: pd.DataFrame(Ann.X)

By default 0, unless scarcity not handled by basis pandas operation
"""

sparseIP= {
        "Healthy_Heart" : 1,
    'Immune_Human' : 0, 
     'Lung' : 1,
     'Pancreas' : 0,
     'Human_Mouse' : 1,
     "Human_Retina": 0
}


In [ ]:
    
# add avg of losses,

import src.model as model


run_config = tf.ConfigProto()

run_config.gpu_options.per_process_gpu_memory_fraction = 0.333
run_config.gpu_options.allow_growth = True

with tf.Session(config = run_config) as sess:

    dreamer = model.scDREAMER(
        sess,
        epoch = epoch_dict[name],
        dataset_name = data_path[name],
        batch = batch_key_dict[name],
        cell_type = cell_type_key_dict[name],
        name = name,
        lr_ae = learning_rate[name]['lr_ae'],
        lr_dis = learning_rate[name]['lr_dis'],
        lr_bc = learning_rate[name]['lr_bc'],
        batch_size = BS[name],
        z_dim = latent_dim[name],
        shuffle_type = shuffle_type_dict[name],
        sparseIP = sparseIP[name]
        )

    dreamer.train_cluster()



2023-09-25 15:18:07.040296: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5367 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:af:00.0, compute capability: 7.5


came in load_gene
updated hvg
here [16 16 16 ... 10 10 10]
Data set to work on:
[[0.         0.         0.8323462  ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.8091273  ... 0.         0.         0.        ]
 ...
 [0.44821838 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(32472, 2000)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(32472, 16)
encoder input shape  Tensor("concat:0", shape=(?, 2016), dtype=float32)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.cast` instead.
decoder input shape  Tensor("concat_2:0", shape=(?, 26), 

2023-09-25 15:18:20.604016: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled


Epoch : [0] ,  a_loss = 524.7845, d_loss: -0.0028 ,  db_loss: 2.5346
Epoch : [1] ,  a_loss = 317.7674, d_loss: -0.0643 ,  db_loss: 2.1999
Epoch : [2] ,  a_loss = 275.6534, d_loss: -0.0606 ,  db_loss: 1.8351
Epoch : [3] ,  a_loss = 490.6587, d_loss: -0.0118 ,  db_loss: 2.9970
Epoch : [4] ,  a_loss = 394.0759, d_loss: -0.0620 ,  db_loss: 1.7072
Epoch : [5] ,  a_loss = 445.1429, d_loss: -0.0077 ,  db_loss: 2.1042
Epoch : [6] ,  a_loss = 446.6374, d_loss: -0.0117 ,  db_loss: 1.6384
Epoch : [7] ,  a_loss = 271.0389, d_loss: -0.0422 ,  db_loss: 0.3001
Epoch : [8] ,  a_loss = 426.5664, d_loss: -0.0174 ,  db_loss: 1.6070
Epoch : [9] ,  a_loss = 414.8620, d_loss: -0.0074 ,  db_loss: 1.4562
Epoch : [10] ,  a_loss = 469.5799, d_loss: -0.0036 ,  db_loss: 2.2595
Epoch : [11] ,  a_loss = 403.5373, d_loss: -0.0391 ,  db_loss: 2.2582
Epoch : [12] ,  a_loss = 217.8143, d_loss: -0.0381 ,  db_loss: 1.6112
Epoch : [13] ,  a_loss = 372.6418, d_loss: -0.0728 ,  db_loss: 1.7148
Epoch : [14] ,  a_loss = 450.6

Epoch : [117] ,  a_loss = 201.8214, d_loss: -0.0092 ,  db_loss: 1.4055
Epoch : [118] ,  a_loss = 403.5962, d_loss: -0.0030 ,  db_loss: 1.8442
Epoch : [119] ,  a_loss = 234.2900, d_loss: -0.0122 ,  db_loss: 1.3501
Epoch : [120] ,  a_loss = 179.9205, d_loss: -0.0078 ,  db_loss: 1.9597
Epoch : [121] ,  a_loss = 355.1060, d_loss: -0.0052 ,  db_loss: 2.3088
Epoch : [122] ,  a_loss = 419.3604, d_loss: -0.0043 ,  db_loss: 1.1014
Epoch : [123] ,  a_loss = 381.3079, d_loss: -0.0041 ,  db_loss: 1.6208
Epoch : [124] ,  a_loss = 224.4867, d_loss: -0.0245 ,  db_loss: 2.9076
Epoch : [125] ,  a_loss = 370.3925, d_loss: -0.0092 ,  db_loss: 1.2785
Epoch : [126] ,  a_loss = 406.5630, d_loss: -0.0208 ,  db_loss: 2.2208
Epoch : [127] ,  a_loss = 354.9062, d_loss: -0.0083 ,  db_loss: 1.9604
Epoch : [128] ,  a_loss = 320.0208, d_loss: -0.0073 ,  db_loss: 1.3542
Epoch : [129] ,  a_loss = 191.3695, d_loss: -0.0369 ,  db_loss: 2.0226
Epoch : [130] ,  a_loss = 375.2763, d_loss: -0.0382 ,  db_loss: 1.9241
Epoch 

Epoch : [233] ,  a_loss = 401.3030, d_loss: -0.1421 ,  db_loss: 1.9802
Epoch : [234] ,  a_loss = 405.3530, d_loss: -0.1074 ,  db_loss: 1.7413
Epoch : [235] ,  a_loss = 387.7766, d_loss: -0.0038 ,  db_loss: 2.4859
Epoch : [236] ,  a_loss = 311.2856, d_loss: -0.0251 ,  db_loss: 1.5061
Epoch : [237] ,  a_loss = 377.5190, d_loss: -0.0022 ,  db_loss: 1.3547
Epoch : [238] ,  a_loss = 262.8679, d_loss: 0.0766 ,  db_loss: 2.9987
Epoch : [239] ,  a_loss = 339.4026, d_loss: -0.0025 ,  db_loss: 2.1336
Epoch : [240] ,  a_loss = 173.4468, d_loss: 1.5172 ,  db_loss: 2.2747
Epoch : [241] ,  a_loss = 223.7342, d_loss: -0.0195 ,  db_loss: 1.5009
Epoch : [242] ,  a_loss = 198.5003, d_loss: -0.0185 ,  db_loss: 2.2588
Epoch : [243] ,  a_loss = 205.9931, d_loss: -0.0190 ,  db_loss: 1.9023
Epoch : [244] ,  a_loss = 339.6279, d_loss: -0.0026 ,  db_loss: 2.1919
Epoch : [245] ,  a_loss = 379.5602, d_loss: -0.0031 ,  db_loss: 2.0308
Epoch : [246] ,  a_loss = 385.1478, d_loss: 0.9807 ,  db_loss: 2.3457
Epoch : [